## 태스크 3: Amazon SageMaker에 모델 배포

이 태스크에서는 Amazon SageMaker를 사용하여 훈련된 기계 학습 모델을 배포하고 예측을 가져옵니다. 먼저 모델 아티팩트를 Amazon S3 버킷으로 내보냅니다. 그런 다음 모델 아티팩트에서 Sagemaker 모델 객체를 생성합니다. 다음으로 Sagemaker 모델, 변환할 데이터의 S3 버킷 위치, 결과를 저장할 S3 버킷 위치를 지정하여 배치 변환 작업을 생성합니다. 그런 다음 배치 변환 작업을 시작하고 진행 상황을 모니터링합니다. 마지막으로 배치 변환 작업이 완료되면 지정된 S3 버킷에서 결과를 검색합니다. 결과에는 훈련된 모델이 입력 데이터에 대해 수행한 추론이 포함됩니다.

**참고:** 도전 태스크를 완료하는 데 문제가 있는 경우 노트북 하단의 <a href="#appendix" target="_self">**부록**</a> 섹션에서 솔루션을 확인할 수 있습니다.

### 태스크 3.1: 환경 설정

환경 설정을 쉽게 시작할 수 있도록 기본 설정 코드가 제공됩니다. 이러한 셀을 먼저 확인한 다음 실행하여 패키지를 설치하고 변수를 생성합니다.

In [ ]:
#라이브러리 가져오기 및 변수 값 설정

import sys
import boto3
import os
import sagemaker
import time

sess = sagemaker.Session()

bucket = sess.default_bucket()
prefix = "lab3/batch-transform"
bucket_path = "s3://{}".format(bucket)

role = sagemaker.get_execution_role()

region = boto3.Session().region_name


### 태스크 3.2: 모델 생성

이 태스크에서는 사전 훈련된 모델을 가져와 S3에 업로드합니다. 그런 다음 SageMaker에서 모델을 생성합니다. 모델은 나중에 SageMaker 배치 변환 작업을 사용하여 배포됩니다.

#### 요구 사항 및 구성

- **S3.Client.upload_file** API를 사용하여 모델 아티팩트와 테스트 데이터를 S3 버킷에 업로드합니다.

**upload_file** 메서드 사용에 대한 자세한 내용은 [Upload a file to an S3 object](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/upload_file.html#upload-file)를 참조하십시오.

In [ ]:
# Amazon S3 버킷에 모델 업로드
from sagemaker.session import Session
from sagemaker.image_uris import retrieve

sagemaker = boto3.client('sagemaker')
s3_client = boto3.client("s3")

#add_upload-model_code_here


#add_upload-dataset_code_here


**참고:** 모델 아티팩트와 테스트 데이터를 S3 버킷에 업로드하는 자세한 코드는 *부록* 섹션의 <a href="#task3-2-1" target="_self">**S3에 모델 및 테스트 데이터 업로드(태스크 3.2)**</a>를 참조하십시오. 

<a id="task3-2-continue"></a>

다음으로 위의 모델 아티팩트를 사용하여 SageMaker에서 모델 객체를 생성합니다.

#### 요구 사항 및 구성

- **SageMaker.Client.create_model** API를 사용하여 SageMaker에서 모델을 생성합니다.
- Request에서 다음 옵션을 사용하십시오.
    - ModelName='telemarketing-model'
    - ExecutionRoleArn=role (SageMaker가 사용하는 이전에 생성된 IAM 역할)
    - 기본 컨테이너의 경우 다음 옵션을 사용하십시오.
        - Image= XGBoost container image URI (retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.5-1'))
        - ModelDataUrl=model_path (The S3 path where the model artifacts are uploaded earlier) 

**SageMaker.Client.create_model** 메서드 사용에 대한 자세한 내용은 [Creating a model in SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_model.html)를 참조하십시오.

In [ ]:
sagemaker = boto3.client('sagemaker')

#add_create-model_code_here

**참고:** 모델 생성 방법에 대한 자세한 코드는 *부록* 섹션의 <a href="#task3-2-2" target="_self">**SageMaker에서 모델 생성(태스크 3.2)**</a>을 참조하십시오. 

<a id="task3-3"></a>

### 태스크 3.3: 배치 변환기 생성

이 태스크에서는 배치 변환기를 생성합니다.

**Transfomer** API 사용에 대한 자세한 내용은 [Using Transformer API](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer)를 참조하십시오.

#### 요구 사항 및 구성

- **sagemaker.transformer.Transformer** API를 사용하여 SageMaker에서 변환기를 초기화합니다.
- Parameters 섹션에서 다음 옵션을 사용합니다.
    - model_name="telemarketing-model"
    - instance_type="ml.m4.xlarge"
    - instance_count=1
    - assemble_with="Line"
    - strategy="MultiRecord"
    - accept="text/csv"
    - output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)

In [7]:
from sagemaker.transformer import Transformer

#add_create-batch-transformer_code_here
transformer = Transformer(
    
)

**참고:** Transformer 객체를 생성하는 방법에 대한 자세한 코드는 *부록* 섹션의 <a href="#task3-3-1" target="_self">**배치 변환기 객체 생성(태스크 3.3)**</a>을 참조하십시오. 

<a id="task3-4"></a>

### 태스크 3.4: 배치 변환 작업 실행

이 태스크에서는 변환 작업을 실행합니다. 훈련 데이터를 전달하여 배치 변환기에서 예측을 가져옵니다.

**Transformer.transform** API 사용에 대한 자세한 내용은 [Using Transform API](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer.transform)를 참조하십시오.

#### 요구 사항 및 구성

- **Transformer.transform** API를 사용하여 SageMaker에서 변환기를 초기화합니다.
- Parameters 섹션에서 다음 옵션을 사용합니다.
    - batch_path(Amazon S3가 이전에 생성한 입력 데이터 위치)
    - content_type="text/csv"
    - split_type="Line"
    - join_source="Input"

In [ ]:
transformer.transform(
 #add_run-batch-transform-job_code_here   
)

transformer.wait()

**참고:** 변환 작업을 실행하는 방법에 대한 자세한 코드는 *부록* 섹션의 <a href="#task3-4-1" target="_self">**배치 변환 작업 실행(태스크 3.4)**</a>을 참조하십시오. 

<a id="task3-5"></a>

### 태스크 3.5: S3 버킷에 저장된 출력 확인

이 태스크에서는 S3에 있는 배치 변환 작업의 출력을 끌어옵니다. 예측 목록이 표시되어야 합니다.

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
!head test_data.csv.out

축하합니다! Amazon SageMaker를 사용하여 훈련된 기계 학습 모델을 성공적으로 배포하고 예측을 가져왔습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 **결론**을 계속 진행합니다.

## 추가 리소스

- [Deploy models for inference](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)
- [Using SageMaker APIs](https://sagemaker.readthedocs.io/en/stable/api/inference/index.html)

<a name="appendix" id="appendix"></a>

## 부록


<a name="task3-2-1" id="task3-2-1"></a>

### S3에 모델 및 테스트 데이터 업로드(태스크 3.2)

In [ ]:
# Amazon S3 버킷에 모델 업로드
from sagemaker.session import Session
from sagemaker.image_uris import retrieve

#session = boto3.Session()
sagemaker = boto3.client('sagemaker')
s3_client = boto3.client("s3")

#upload-model
s3_client.upload_file(Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz")  
model_path = f"s3://{bucket}/{prefix}/models/model.tar.gz"
print(model_path)

#upload-dataset
s3_client.upload_file(Filename="test_data.csv", Bucket=bucket, Key=f"{prefix}/test_data.csv", ExtraArgs={"ContentType": "text/csv;charset=utf-8"})
batch_path = f"s3://{bucket}/{prefix}/test_data.csv"
print(batch_path)

이 실습을 계속 진행하려면 <a href="#task3-2-continue" target="_self">태스크 3.2</a>로 돌아갑니다.

<a name="task3-2-2" id="task3-2-2"></a>

### SageMaker에서 모델 생성(태스크 3.2)

In [ ]:
sagemaker = boto3.client('sagemaker')

# 컨테이너 이미지 검색
image = retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.5-1')

primary_container = {"Image": image, "ModelDataUrl": model_path}

# 모델 설정
create_model_response = sagemaker.create_model(
    ModelName='telemarketing-model', ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

이 실습을 계속 진행하려면 <a href="#task3-3" target="_self">태스크 3.3</a>으로 돌아갑니다.

<a name="task3-3-1" id="task3-3-1"></a>

### 배치 변환기 객체 생성(태스크 3.3)

In [ ]:
from sagemaker.transformer import Transformer

#create-batch-transformer
transformer = Transformer(
    model_name="telemarketing-model",
    instance_type="ml.m4.xlarge",
    instance_count=1,
    assemble_with="Line",
    strategy="MultiRecord",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/test".format(bucket, prefix)
)

이 실습을 계속 진행하려면 <a href="#task3-4" target="_self">태스크 3.4</a>로 돌아갑니다.

<a name="task3-4-1" id="task3-4-1"></a>

### 배치 변환 작업 실행(태스크 3.4)


In [ ]:
#run-batch-transform-job
transformer.transform(batch_path, content_type="text/csv", split_type="Line", join_source="Input")

transformer.wait()

이 실습을 계속 진행하려면 <a href="#task3-5" target="_self">태스크 3.5</a>로 돌아갑니다.